In [ ]:
import sys
import os
import numpy as np
import napari
%matplotlib inline
from matplotlib import pyplot as plt  # graphic library, for plots
#import seaborn as sns
import numba as nb

from ipywidgets import interact
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure, show
from bokeh.models import CustomJS, ColumnDataSource, Slider, CDSView, BooleanFilter, CustomJS, RangeSlider, Label, RangeTool
from bokeh.layouts import column, grid, gridplot, layout
from bokeh.sampledata.stocks import AAPL

In [ ]:
from metavision_core.event_io.raw_reader import RawReader
from metavision_core.event_io.py_reader import EventDatReader
from metavision_core.event_io import EventsIterator

In [ ]:
import filedialogs

### Get Path

In [ ]:
path = filedialogs.gui_fname('example_data/')
path = path.decode('ascii')

print(path)

### Get total number of events

In [ ]:
def get_total_event_number(path):
    mv_iterator = EventsIterator(path, delta_t=1000000)
    global_counter = 0  # This will track how many events we processed
    for evs in mv_iterator:
        counter = evs.size  # Local counter
        global_counter += counter  # Increase global counter
    return global_counter

In [ ]:
total_event_number = get_total_event_number(path)
print('Total event number: ', total_event_number)

### Load everything

In [ ]:
record_raw = RawReader(path, max_events=total_event_number+1)
events = record_raw.load_n_events(total_event_number+1)
print(events.size)

### Helper functions

In [ ]:
def calculate_bins(events, dt=1000):
    # calculate events for timestep dt in us
    min_timepoint = events['t'].min()
    max_timepoint = events['t'].max()
    interval_us = max_timepoint-min_timepoint
    timepoints = int(interval_us / (dt))
    bins = np.linspace(min_timepoint, max_timepoint, timepoints)
    return bins

def calculate_bin_number(events, dt=1000):
    # calculate events for timestep dt in us
    min_timepoint = events['t'].min()
    max_timepoint = events['t'].max()
    timepoints = int(np.floor((max_timepoint-min_timepoint) / (dt)))
    return timepoints

@nb.njit
def inside_circular_roi(x, y, x_center, y_center, r):
    ''' Checks whether a point with coordinates x and y is within 
        circle with radius r around x_center,y_center '''
    return np.power(x-x_center,2)+np.power(y-y_center,2) <= np.power(r,2)

@nb.njit
def inside_rectanglular_roi(x,y,x0,y0,x1,y1):
    ''' Checks whether a point with coordinates x and y is within a 
        rectangle between x0,y0 (top left corner) and x1,y1 (bottom right corner) '''
    return x >= x0 and x <= x1 and y >= y0 and y <= y1

def is_on_event(event):
    return event[2]==1

@nb.njit
def filter_array(arr, condition):
    result = np.empty_like(arr)
    j = 0
    for i in range(arr.size):
        if condition(arr[i]):
            result[j] = arr[i]
            j += 1
    return result[:j].copy()

@nb.njit
def events_inside_circular_roi(events, x_center, y_center, radius):
    result = np.empty_like(events)
    j = 0
    for i in range(events.size):
        if inside_circular_roi(events[i]['x'],events[i]['y'],x_center,y_center, radius):
            result[j] = events[i]
            j += 1
    return result[:j].copy()

@nb.njit
def events_inside_rectangular_roi(events, x0,y0,x1,y1):
    result = np.empty_like(events)
    j = 0
    for i in range(events.size):
        if inside_rectanglular_roi(events[i]['x'],events[i]['y'],x0,y0,x1,y1):
            result[j] = events[i]
            j += 1
    return result[:j].copy()

@nb.njit
def split_events(events):
    on_events = np.empty_like(events)
    off_events = np.empty_like(events)
    on_counter = 0
    off_counter = 0
    
    for i in range(events.size):
        if events[i][2]==1:
            on_events[on_counter] = events[i]
            on_counter += 1
        else: 
            off_events[off_counter] = events[i]
            off_counter += 1
   
    return on_events[:on_counter].copy(),off_events[:off_counter].copy()

def events_to_stack(events, dt=1000, imagesize=(720, 1280)):
    height, width = imagesize
    min_timepoint = events['t'].min()
    max_timepoint = events['t'].max()
    
    timepoints = int(np.ceil((max_timepoint - min_timepoint)/dt))    
    
    # generate a numpy array with the width 
    eventstack = np.zeros((timepoints,height,width,2), dtype='uint8')
    
    for event in events:
        x_pos = event[1]
        y_pos = event[0]
        timepoint = np.floor_divide((event[3]-min_timepoint),dt)
        if event[2] == 0:
            if eventstack[timepoint, x_pos, y_pos, 1] < 255:
                eventstack[timepoint, x_pos, y_pos, 1] += 1
        else: 
            if eventstack[timepoint, x_pos, y_pos, 0] < 255:
                eventstack[timepoint, x_pos, y_pos, 0] += 1       
    
    return eventstack 

### Visualize all events

In [ ]:
def viz_events(events, height, width):
    img = np.full((height, width, 3), 128, dtype=np.uint8)
    img[events['y'], events['x']] = 255 * events['p'][:, None]
    return img

In [ ]:
height, width = record_raw.get_size()
# load the next 50 ms worth of events
# events = record_raw.load_delta_t(50000)
im = viz_events(events, height, width)

plt.imshow(im)
plt.tight_layout()

### Crop events in space

In [ ]:
inside_events = events_inside_rectangular_roi(events, 350, 0, 1050, 700)
on_events, off_events = split_events(inside_events)

In [ ]:
height, width = record_raw.get_size()

# load the next 50 ms worth of events
# events = record_raw.load_delta_t(50000)
im = viz_events(inside_events, height, width)

plt.imshow(im)
plt.tight_layout()

### Crop Events in time

In [ ]:
starttime = 2.25e8
endtime = 2.34e8

In [ ]:
on_events_crop_filter = [i[3] >= starttime and i[3] <= endtime for i in on_events]
on_events_crop = on_events[on_events_crop_filter]

In [ ]:
off_events_crop_filter = [i[3] >= starttime and i[3] <= endtime for i in off_events]
off_events_crop = off_events[off_events_crop_filter]

In [ ]:
inside_events_crop_filter = [i[3] >= starttime and i[3] <= endtime for i in inside_events]
inside_events_crop = inside_events[inside_events_crop_filter]

### Pickle key objects

In [ ]:
import pickle

In [ ]:
def storeData():
    db = {  'on_events_crop':     on_events_crop,
            'off_events_crop':    off_events_crop,
            'inside_events_crop': inside_events_crop,        
    }
    
    dbfile = open('myExampleEvents', 'ab')
    pickle.dump(db,dbfile)
    dbfile.close()

In [ ]:
def loadData():
    dbfile = open('myExampleEvents', 'rb')    
    db = pickle.load(dbfile)
    dbfile.close()
    return db

In [ ]:
storeData()

In [ ]:
db = loadData()
on_events_crop = db['on_events_crop']
off_events_crop = db['off_events_crop']
inside_events_crop = db['inside_events_crop']

### Quickstart here

In [1]:
import sys
import os
import numpy as np
import napari
%matplotlib inline
from matplotlib import pyplot as plt  # graphic library, for plots
#import seaborn as sns
import numba as nb
import pickle

from ipywidgets import interact
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure, show
from bokeh.models import CustomJS, ColumnDataSource, Slider, CDSView, BooleanFilter, CustomJS, RangeSlider, Label, RangeTool
from bokeh.layouts import column, grid, gridplot, layout
from bokeh.sampledata.stocks import AAPL

In [2]:
def loadData():
    dbfile = open('myExampleEvents', 'rb')    
    db = pickle.load(dbfile)
    dbfile.close()
    return db

def calculate_bin_number(events, dt=1000):
    # calculate events for timestep dt in us
    min_timepoint = events['t'].min()
    max_timepoint = events['t'].max()
    timepoints = int(np.floor((max_timepoint-min_timepoint) / (dt)))
    return timepoints

In [3]:
db = loadData()
on_events_crop = db['on_events_crop']
off_events_crop = db['off_events_crop']
inside_events_crop = db['inside_events_crop']

### Event histogram overview

In [4]:
bins = calculate_bin_number(off_events_crop, dt=1000)

In [5]:
on_hist, on_edges = np.histogram(on_events_crop['t'], density=False, bins=bins)
off_hist, off_edges = np.histogram(off_events_crop['t'], density=False, bins=bins)

In [6]:
output_notebook()

Loading BokehJS ...

In [7]:
source = ColumnDataSource(data=dict(edges=on_edges[:-1], on_hist=on_hist, off_hist=off_hist))

p = figure(height=400, width=950, 
           x_axis_location="below", background_fill_color="#efefef",x_range=(on_edges[1000], on_edges[2000]))

p.quad(top=off_hist, bottom=0, left=on_edges[:-1], right=on_edges[1:],
         fill_color="darkred", line_width=0, legend_label="Off Events")
p.quad(top=on_hist+off_hist, bottom=off_hist, left=on_edges[:-1], right=on_edges[1:],
         fill_color="royalblue", line_width=0, legend_label="On Events")
p.y_range.start = 0
p.xaxis.axis_label = "t (us)"
p.yaxis.axis_label = "Events"
p.legend.location = "top_right"

select = figure(title="Event Selection",
                height=230, width=950, y_range=p.y_range, y_axis_type=None, background_fill_color="#efefef")

range_tool = RangeTool(x_range=p.x_range)
range_tool.overlay.fill_color = "navy"
range_tool.overlay.fill_alpha = 0.2

select.line('edges', 'on_hist', source=source)
select.ygrid.grid_line_color = None
select.add_tools(range_tool)

show(column(p, select))

### Let's start with OpenGL

In [8]:
%gui qt5
from PyQt5.Qt import QApplication
import qdarkstyle
import pyqtgraph as pg
from pyqtgraph import functions as fn
#from pyqtgraph.Qt import QtCore, QtGui
import pyqtgraph.opengl as gl
import numpy as np

In [9]:
# start qt event loop
_instance = QApplication.instance()
if not _instance:
    _instance = QApplication([])
app = _instance

In [10]:
w = gl.GLViewWidget()
w.show()

In [ ]:
# Remove all items
w.clear()

In [ ]:
app.quit()

In [114]:
w.setWindowTitle('pyqtgraph example: GLScatterPlotItem')
w.setCameraPosition(distance=20)

g = gl.GLGridItem()
w.addItem(g)

### Process events

In [118]:
x = on_events_crop['x'].astype('float')
y = on_events_crop['y'].astype('float')
# The division factor is arbitrary
x = (x-700)/40
y = (y-350)/40

In [119]:
# convert time axis to ms
t = -((on_events_crop['t']-on_events_crop['t'][0])/500000).astype('float')
t = t - t[-1]

In [120]:
pos = np.transpose(np.stack([x, y, t])).astype('float')

In [121]:
on_eventplot = gl.GLScatterPlotItem(pos=pos,color=(50,125,255,127), size=1)
w.addItem(on_eventplot)

#### Off plot

In [137]:
x = off_events_crop['x'].astype('float')
y = off_events_crop['y'].astype('float')
# The division factor is arbitrary
x = (x-700)/40
y = (y-350)/40

In [138]:
# convert time axis to ms
t = -((off_events_crop['t']-on_events_crop['t'][0])/500000).astype('float')
t = t - t[-1]

In [139]:
pos = np.transpose(np.stack([x, y, t])).astype('float')

In [140]:
off_eventplot = gl.GLScatterPlotItem(pos=pos,color=(150,0,0,127), size=1)
w.addItem(off_eventplot)

In [ ]:
w.cameraParams()

In [ ]:
on_eventplot.setData(color=(0,0,150,127))

### Remote items

In [ ]:
w.removeItem(on_eventplot)

### Event filtering

* Iterate through all events
* timerange in us
* x_offset, y_offset
* nearest neighbour 

In [20]:
len(off_events_crop)

336965

In [21]:
type(off_events_crop['x'][0])

numpy.uint16

In [ ]:
np.mod(9,9)

In [ ]:
np.seterr(over='ignore')

In [24]:
xy_radius_pix = 10
delta_t_us = 100

# @nb.njit
def outlier_event_removal(events):
    total_events = len(events)
    fivepercent = int(total_events / 20)
    print('We have: ', total_events, 'events')
    processed_events = np.empty(shape=(0,),dtype=off_events_crop.dtype)
    index = 0
          
    for event in events:
        #print('Current index: ', index)
        if np.mod(index, fivepercent) == 0:
            print('Another 5% done')
        index += 1
        timepoint = event[3]
        x_pos = event[0]
        y_pos = event[1]
        offset_index = 0
        if index == total_events:
            continue
        else:
            next_timepoint = events[index][3] 
            
        while next_timepoint - timepoint < delta_t_us:
            offset_index += 1
            # print('Offset index: ', offset_index)
            next_index = index+offset_index
            if index + offset_index == total_events:
                break
            else:
                next_event = events[next_index]
                next_timepoint = next_event[3]
                next_x_pos = next_event[0].astype('int')
                next_y_pos = next_event[1].astype('int')
                                
                if np.abs(x_pos - next_x_pos) < xy_radius_pix:
                    if np.abs(y_pos - next_y_pos) < xy_radius_pix:
                        processed_events = np.append(processed_events, event)
                    else:
                        continue
                else:
                    continue   
                    
    return processed_events

In [ ]:
np.abs(-9)

In [25]:
processed_events = outlier_event_removal(off_events_crop)

We have:  336965 events
Another 5% done
Another 5% done
Another 5% done
Another 5% done
Another 5% done
Another 5% done
Another 5% done
Another 5% done
Another 5% done
Another 5% done
Another 5% done
Another 5% done
Another 5% done
Another 5% done
Another 5% done
Another 5% done
Another 5% done
Another 5% done
Another 5% done
Another 5% done
Another 5% done


In [26]:
len(processed_events)

880475

In [40]:
processed_events[0:10]

array([(1008, 393, 0, 225491557), ( 593,  41, 0, 225677651),
       ( 588,  47, 0, 226066074), ( 590,  47, 0, 226066173),
       ( 597,  28, 0, 226066650), ( 583,  38, 0, 226066652),
       ( 583,  38, 0, 226066652), ( 583,  38, 0, 226066652),
       ( 585,  43, 0, 226066710), ( 603,  37, 0, 226066744)],
      dtype=[('x', '<u2'), ('y', '<u2'), ('p', '<i2'), ('t', '<i8')])

In [41]:
unique_processed_events = np.unique(processed_events)

In [42]:
len(unique_processed_events)

220275

In [30]:
def storeData():
    db = {  'filtered_data':     processed_events,
                }
    
    dbfile = open('myFilteredData', 'ab')
    pickle.dump(db,dbfile)
    dbfile.close()

In [31]:
storeData()

In [33]:
w.removeItem(off_eventplot)

In [34]:
x = processed_events['x'].astype('float')
y = processed_events['y'].astype('float')
# The division factor is arbitrary
x = (x-700)/40
y = (y-350)/40

In [35]:
# convert time axis to ms
t = -((processed_events['t']-processed_events['t'][0])/500000).astype('float')
t = t - t[-1]

In [36]:
pos = np.transpose(np.stack([x, y, t])).astype('float')

In [37]:
off_eventplot = gl.GLScatterPlotItem(pos=pos,color=(150,0,0,127), size=1)
w.addItem(off_eventplot)

### Explore average trajectory

In [141]:
x = on_events_crop['x'].astype('float')
y = on_events_crop['y'].astype('float')
# The division factor is arbitrary
x = (x-700)/40
y = (y-350)/40

In [142]:
# convert time axis to ms
t = -((on_events_crop['t']-on_events_crop['t'][0])/500000).astype('float')
t = t - t[-1]

In [143]:
pos = np.transpose(np.stack([x, y, t])).astype('float')

In [48]:
pos[0:10]

array([[ 6.9     , -3.525   , 17.996218],
       [ 8.225   , -3.425   , 17.99573 ],
       [ 6.475   , -4.775   , 17.99542 ],
       [ 7.475   ,  0.425   , 17.995056],
       [ 0.475   ,  7.55    , 17.99445 ],
       [-1.65    ,  8.425   , 17.993718],
       [-2.475   ,  8.45    , 17.99244 ],
       [-6.825   , -8.4     , 17.991384],
       [ 8.325   , -2.35    , 17.99008 ],
       [-6.75    , -4.325   , 17.9847  ]])

In [32]:
pos_t = np.transpose(pos)

In [37]:
np.mean(pos_t[2][0:1000])

17.479406925999996

In [22]:
len(x)

257495

In [23]:
len(t)

257495

In [42]:
np.ndarray([1,1,1])

array([[[1.19175502e-311]]])

In [144]:
def create_average_trajectory(events, windowsize):
    max_index = len(events)
    events_t = np.transpose(events)
    start_index = 0
    end_index = windowsize
    averaged_events = np.empty(shape=(0,3),dtype=events.dtype)
    while end_index <= max_index and start_index <= end_index:
        x_mean = np.mean(events_t[0][start_index:end_index])
        y_mean = np.mean(events_t[1][start_index:end_index])
        t_mean = np.mean(events_t[2][start_index:end_index])
        avg_event = np.array([[x_mean, y_mean, t_mean]])
        start_index += windowsize
        end_index += windowsize
        averaged_events = np.concatenate((averaged_events, avg_event))
    print('Number of averaged output events: ', len(averaged_events))
    return averaged_events
    
    

In [148]:
avg_events = create_average_trajectory(pos, 200)

Number of averaged output events:  1287


In [149]:
on_eventline2 = gl.GLLinePlotItem(pos=avg_events,color=(80,80,0,127), width=2, mode='line_strip')
w.addItem(on_eventline2)

In [ ]:
w.removeItem(on_eventline)

In [136]:
w.clear()

### Notes
* during the tail flicks, averaging works really well
* in the coasting phase, the event rate drops to background levels
* as a result, the average is pulled into the center 
* drifting debris (moved by water motion)
* possible solutions: 
    * system should have a memory where the fish was?
    * dual approach:
        * if eventrate goes above a certain level, just average the position
        * if event rate is below a certain level, revert to memory and stay there?
    * noise removal approach -> remove outlier noise first, then average
         * requires superfast noise detection...